In [54]:
import pandas as pd

In [55]:
arts = pd.read_csv('shared_articles.csv')
ints = pd.read_csv('users_interactions.csv')

In [56]:
arts.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


In [57]:
#Задание 6.1
#Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. 
#Сколько таких объектов в получившейся таблице?
arts = arts[arts['eventType'] == 'CONTENT SHARED']
arts.shape

(3047, 13)

In [58]:
ints.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [59]:
ints.personId = ints.personId.astype(str)
ints.contentId = ints.contentId.astype(str)
arts.contentId = arts.contentId.astype(str)

In [60]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [61]:
#Задание 6.2

#Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй 
# (в соответствии с приведёнными выше весами). 
# Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.
ints['rating'] = ints['eventType'].apply(lambda x: event_type[x])
round(ints['rating'].mean(),2)

1.24

In [62]:
#Задание 6.3

#Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, 
# которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?
data = ints.groupby('personId')['contentId'].nunique()
data = list(data[data>=5].index)
len(data)

1140

In [63]:
ints['trust_person'] = ints['personId'].apply(lambda x: 1 if x in data else 0)
ints = ints[ints['trust_person'] == 1]
ints = ints.drop('trust_person', axis=1)

In [64]:
#Задание 6.4

#Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей 
# (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?
ints.shape[0]

69868

Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:



In [65]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [66]:
#Задание 6.5

#Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя 
# с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

#Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

In [67]:
arts_rating_df = pd.pivot_table(ints, index=['personId','contentId'], 
                                values=['timestamp','rating'], 
                                aggfunc={'timestamp':'max','rating':'sum'}).reset_index()

arts_rating_df['rating'] = arts_rating_df['rating'].apply(smooth_user_preference)

In [68]:
arts_rating_df['timestamp'].mean()

1470605340.0403006

In [69]:
#Задание 6.6

#Разделите данные на обучающую и тестовую выборки, выбрав в 
# качестве временной отсечки значение 1475519545. 
# Значение отсечки включите в тестовую выборку. Сколько объектов попало в обучающую выборку?
train_df = arts_rating_df[arts_rating_df['timestamp']<1475519545]
test_df = arts_rating_df[arts_rating_df['timestamp']>=1475519545]
train_df.shape[0]

29325

Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.

In [70]:
import numpy as np

In [71]:
train_df

,personId,contentId,rating,timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054096
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


In [72]:
final_df = (
    train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


In [73]:
#Задание 6.7

#Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! 
# Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

#Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней 
# (используя только обучающую выборку). Выберите ID самой популярной статьи:
train_df.groupby('contentId')['rating'].sum().sort_values(ascending=False)

contentId
-6783772548752091658    231.177195
-133139342397538859     228.024567
-8208801367848627943    189.937683
8224860111193157980     186.044680
7507067965574797372     179.094002
                           ...    
-1861959619649725543      1.000000
4119190424078847945       1.000000
-1852234953549915446      1.000000
4106497696154898573       1.000000
1729892529360390310       1.000000
Name: rating, Length: 2366, dtype: float64

Теперь необходимо сформировать рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.

In [74]:
#Задание 6.8

#Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). 
# После этого усредните результат по всем пользователям.

#Для вычисления precision@10 воспользуйтесь следующей функцией:

def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
#Итоговый результат округлите до трёх знаков после точки-разделителя.

#В этом задании необходимо использовать данные, полученные в задании 6.6 (для которых приведён скриншот выше).

In [84]:
recomend_list = list(train_df.groupby('contentId')['rating'].sum().sort_values(ascending=False).index)[:10]
final_df['recomendation'] = final_df['true_train'].apply(lambda y: list(set(recomend_list).difference(set(y))))
precision('recomendation')

0.004285989399957339

ТК У МЕНЯ НЕ СОВПАЛ ПОСЛЕДНИЙ ОТВЕТ, ПОСМОТРИМ НА ЭТАЛОН:

In [85]:
articles_df = pd.read_csv('shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.shape

(3047, 13)

In [86]:
interactions_df = pd.read_csv('users_interactions.csv')
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])
interactions_df['eventStrength'].mean()

1.2362885828078327

In [87]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

print(len(users_with_enough_interactions_df))

1140


In [88]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]
print(interactions_from_selected_users_df.shape)

(69868, 9)


In [90]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df['last_timestamp'].mean()

1470587338.35212